In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import csv
from datetime import datetime, timedelta

# 初始化 WebDriver（不使用 headless 模式）
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# 定義目標 URL
url = "https://maps.app.goo.gl/1iwhfCzfkEHDf98DA"
driver.get(url)
time.sleep(2)

# 抓取景點名稱
try:
    title_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf"))
    )
    title = title_element.text
    print(f"景點名稱: {title}")
except Exception as e:
    print("無法抓取景點名稱:", e)
    driver.quit()

# 確保評論區按鈕生成
scroll_attempts = 0
button_found = False

while scroll_attempts < 5 and not button_found:
    driver.execute_script("window.scrollBy(0, 300);")  # 每次滾動300像素
    time.sleep(random.uniform(1, 2))  # 增加停頓時間隨機性
    try:
        # 檢查評論按鈕是否已經可見
        comment_button = driver.find_element(By.CSS_SELECTOR, "button.HHrUdb")
        button_found = True
    except Exception:
        scroll_attempts += 1

if not button_found:
    print("無法找到評論區按鈕，請檢查網頁結構或滾動邏輯。")
    driver.quit()

# 點擊評論區按鈕
scroll_attempts = 0
while scroll_attempts < 3:
    driver.execute_script("window.scrollBy(0, document.body.scrollHeight/3);")
    time.sleep(random.uniform(1, 2))
    scroll_attempts += 1
driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
time.sleep(random.uniform(1, 2))

# 點擊評論區按鈕
try:
    comment_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.HHrUdb"))
    )
    comment_button.click()
    time.sleep(random.uniform(1, 2))
except Exception as e:
    print("無法找到評論區按鈕:", e)
    driver.quit()

# 更改排序為"最新"
try:
    sort_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-value='排序']"))
    )
    sort_button.click()
    time.sleep(1)

    latest_button = driver.find_elements(By.CSS_SELECTOR, "div.mLuXec")[1]
    driver.execute_script("arguments[0].scrollIntoView();", latest_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", latest_button)  # 使用 JavaScript 點擊
    time.sleep(2)
except Exception as e:
    print("無法更改排序為最新:", e)
    driver.quit()

# 滑動頁面並抓取評論
def parse_date(date_str):
    if "天前" in date_str:
        days = int(date_str.replace("天前", ""))
        return (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")
    elif "週前" in date_str:
        weeks = int(date_str.replace("週前", ""))
        return (datetime.now() - timedelta(weeks=weeks)).strftime("%Y-%m-01")
    elif "個月前" in date_str:
        months = int(date_str.replace("個月前", ""))
        now = datetime.now()
        month = (now.month - months - 1) % 12 + 1
        year = now.year - ((now.month - months - 1) // 12)
        return f"{year}-{month:02d}-01"
    elif "年前" in date_str:
        years = int(date_str.replace("年前", ""))
        return f"{datetime.now().year - years}-01-01"
    return date_str

comments = []
loaded_reviews = 0
target_reviews = 50000  # 設定目標評論數

while loaded_reviews < target_reviews:
    reviews = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")

    for review in reviews:
        try:
            review_id = review.get_attribute("data-review-id")
            rating_element = review.find_element(By.CSS_SELECTOR, "span.kvMYJc")
            rating = rating_element.get_attribute("aria-label").replace(" 顆星", "")

            # 處理評論內容
            if review.find_elements(By.CSS_SELECTOR, "button.kyuRq.WOKzJe"):
                review.find_element(By.CSS_SELECTOR, "button.kyuRq.WOKzJe").click()
                time.sleep(1)

            if review.find_elements(By.CSS_SELECTOR, "button.w8nwRe.kyuRq"):
                review.find_element(By.CSS_SELECTOR, "button.w8nwRe.kyuRq").click()
                time.sleep(1)

            comment_element = review.find_element(By.CSS_SELECTOR, "span.wiI7pd")
            comment_content = comment_element.text

            date_element = review.find_element(By.CSS_SELECTOR, "span.rsqaWe")
            date_time = parse_date(date_element.text)

            comments.append({
                "ID": review_id,
                "Rating": rating,
                "Comment": comment_content,
                "Date": date_time
            })
            loaded_reviews += 1
            print(f"正在處理第{loaded_reviews}則評論")

            if loaded_reviews >= target_reviews:
                break
        except Exception as e:
            print("評論抓取錯誤:", e)

    # 滑動頁面加載更多評論
    if loaded_reviews < target_reviews:
        driver.execute_script("arguments[0].scrollIntoView();", reviews[-1])
        time.sleep(random.uniform(1.5, 2.5))

# 保存評論到 CSV
csv_file = "comments_google_map_dtf_101.csv"
with open(csv_file, "w", newline="", encoding="utf-8-sig") as file:
    writer = csv.DictWriter(file, fieldnames=["ID", "Rating", "Comment", "Date"])
    writer.writeheader()
    writer.writerows(comments)

print(f"成功抓取 {len(comments)} 則評論並保存至 {csv_file}")

driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import csv

# 初始化 WebDriver
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# 定義目標 URL
url = "https://maps.app.goo.gl/VyfjQnfxF6KVyVnK8"
driver.get(url)
time.sleep(2)

# 抓取景點名稱
try:
    title_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf"))
    )
    title = title_element.text
    print(f"景點名稱: {title}")
except Exception as e:
    print("無法抓取景點名稱:", e)
    driver.quit()

# 確保評論區按鈕生成
scroll_attempts = 0
button_found = False

while scroll_attempts < 5 and not button_found:
    driver.execute_script("window.scrollBy(0, 300);")
    time.sleep(random.uniform(1, 2))
    try:
        comment_button = driver.find_element(By.CSS_SELECTOR, "button.HHrUdb")
        button_found = True
    except Exception:
        scroll_attempts += 1

if not button_found:
    print("無法找到評論區按鈕，請檢查網頁結構或滾動邏輯。")
    driver.quit()

# 點擊評論區按鈕
try:
    comment_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.HHrUdb"))
    )
    comment_button.click()
    time.sleep(random.uniform(1, 2))
except Exception as e:
    print("無法找到評論區按鈕:", e)
    driver.quit()

# 更改排序為"最新"
try:
    sort_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-value=\'排序\']"))
    )
    sort_button.click()
    time.sleep(1)

    latest_button = driver.find_elements(By.CSS_SELECTOR, "div.mLuXec")[1]
    driver.execute_script("arguments[0].scrollIntoView();", latest_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", latest_button)
    time.sleep(2)
except Exception as e:
    print("無法更改排序為最新:", e)
    driver.quit()

# 滑動頁面並抓取評論
comments = []
unique_review_ids = set()  # 用於檢查評論 ID 的唯一性
scroll_attempts = 0
max_scroll_attempts = 10  # 設定最多滑動次數
previous_height = driver.execute_script("return document.body.scrollHeight")  # 初始化頁面高度

while scroll_attempts < max_scroll_attempts:
    reviews = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")

    # 調試輸出當前找到的評論數量
    print(f"當前找到的評論數量: {len(reviews)}")

    for review in reviews:
        try:
            review_id = review.get_attribute("data-review-id")
            if review_id in unique_review_ids:
                continue  # 跳過已抓取的評論

            unique_review_ids.add(review_id)

            rating_element = review.find_element(By.CSS_SELECTOR, "span.kvMYJc")
            rating = rating_element.get_attribute("aria-label").replace(" 顆星", "")

            if review.find_elements(By.CSS_SELECTOR, "button.kyuRq.WOKzJe"):
                review.find_element(By.CSS_SELECTOR, "button.kyuRq.WOKzJe").click()
                time.sleep(1)

            if review.find_elements(By.CSS_SELECTOR, "button.w8nwRe.kyuRq"):
                review.find_element(By.CSS_SELECTOR, "button.w8nwRe.kyuRq").click()
                time.sleep(1)

            comment_element = review.find_elements(By.CSS_SELECTOR, "span.wiI7pd")
            comment_content = comment_element[0].text if comment_element else ""

            date_element = review.find_element(By.CSS_SELECTOR, "span.rsqaWe")
            date_time = date_element.text

            comments.append({
                "ID": review_id,
                "Rating": rating,
                "Comment": comment_content,
                "Date": date_time
            })
        except Exception as e:
            print("評論抓取錯誤:", e)

    # 滑動頁面加載更多評論
    if reviews:
        driver.execute_script("arguments[0].scrollIntoView();", reviews[-1])
        print("滑動至最後一條評論，等待加載更多內容。")
        time.sleep(random.uniform(3, 5))  # 延長停留時間以加載更多評論

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == previous_height:
        scroll_attempts += 1
        print(f"無法加載更多評論，第 {scroll_attempts} 次嘗試。")
        if scroll_attempts >= max_scroll_attempts:
            print("已達到最大滑動嘗試次數，停止抓取。")
            break
    else:
        scroll_attempts = 0  # 重置嘗試次數
    previous_height = new_height

# 保存評論到 CSV
csv_file = "comments_google_map.csv"
with open(csv_file, "w", newline="", encoding="utf-8-sig") as file:
    writer = csv.DictWriter(file, fieldnames=["ID", "Rating", "Comment", "Date"])
    writer.writeheader()
    writer.writerows(comments)

print(f"成功抓取 {len(comments)} 則評論並保存至 {csv_file}")

driver.quit()
